# Obtaining the Postal Codes, Boroughs, and Neighborhoods of Toronto



## Import Libraries needed for assignment

In [1]:
import os.path

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

from bs4 import BeautifulSoup
import re

print('Libraries imported.')

Libraries imported.


# TASK 1 Grabbing data off wiki page



## Decided to save webpage to an html file in the event the wiki page gets editted incorrectly by someone.

If the html file exists, then the file is read into `wiki_html`

If the html file does not exist, then the webpage is scraped(in its current version) to `wiki_html`. Which then gets saved to a html file.

Finally `wiki_html` is used to create BeautifulSoup object.

In [2]:
if os.path.exists('wiki.html'):   
    with open('wiki.html','rb') as html:
        wiki_html = html.read()
else:
    wiki_page = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
    wiki_response = requests.get(wiki_page)
    wiki_html = wiki_response.content
    with open('wiki.html','wb') as html: #writes as binary file
        html.write(wiki_html)

wiki_soup = BeautifulSoup(wiki_html, 'html.parser')


## Using the BeautifulSoup object (`wiki_soup`) to create pandas dataframe with columns of PostalCode, Borough, Neighborhood

First, the table with the required data is located. 

For each assigned Postal Code:

The data (Postal Code, Borough, Neighborhood) for each assigned Postal Code is put into a dictionary (`cell`).

The `cell['Borough']` and `cell['Neighborhood']` lines grabs all the Boroughs and Neighborhoods for the current Postal Code.

Then the `cell` dictionary is appended to a list which will be converted into a pandas dataframe.

Final step is too replace some of the Boroughs with easier to read names


In [3]:
wiki_table_contents = []
wiki_table = wiki_soup.find('table') # Locates table with required data

for row in wiki_table.findAll('td'): # Goes through each of the table's cells and extracts the data associated with each assigned Postal Code
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0] # "/ ".join(elem for elem in re.findall(r"(.*?)(?:\(.*?\)|$)", row.span.text)).replace('/ ','/').strip('/ ')         # 
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ') #  ", ".join([elem for elem in re.findall('\((.*?)\)', row.span.text)]).replace(' /',',').strip(' ')  #
        wiki_table_contents.append(cell)
        
toronto_df=pd.DataFrame(wiki_table_contents) 
toronto_df['Borough'] = toronto_df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})


In [4]:
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


## Tiding up list a bit

In [5]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))

In [6]:
neighbours = toronto_df.groupby(['PostalCode', 'Borough'])
toronto_df = neighbours.apply(neighborhood_list).reset_index(name='Neighborhood')
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Shape of `toronto_df`

In [7]:
toronto_df.shape

(103, 3)

# Task 2 Obtaining Coordinates for Toronto's neighborhoods

Using the provided Geospatial_Coordinates.csv, the csv is read into the `coords_df` dataframe.

The `Postal Code` column is renamed to `PostalCode` in order to merge with the `toronto_df` dataframe to create the new combined dataframe : `torontoCoords_df'



In [8]:
coords_df = pd.read_csv('Geospatial_Coordinates.csv').rename(columns={'Postal Code': 'PostalCode'})
coords_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merging dataframes based off the PostalCode column. 

In [9]:
torontoCoords_df = pd.merge(toronto_df,coords_df, on='PostalCode')
torontoCoords_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Task 3 Analzying the **Toronto** neighboorhoods

In [10]:
print('Toronto has {} boroughs and {} neighborhoods.\n'.format(len(toronto_df['Borough'].unique()),toronto_df.shape[0]))
print("The {} Boroughs are:\n {}".format(len(toronto_df['Borough'].unique()), (toronto_df['Borough'].unique())))

Toronto has 15 boroughs and 103 neighborhoods.

The 15 Boroughs are:
 ['Scarborough' 'North York' 'East York' 'East Toronto'
 'East York/East Toronto' 'Central Toronto' 'Downtown Toronto'
 'Downtown Toronto Stn A' 'York' 'West Toronto' "Queen's Park"
 'Mississauga' 'East Toronto Business' 'Etobicoke' 'Etobicoke Northwest']


## First, I will show all the neighboorhoods within  `Toronto`.



### Begin by grabbing the general coordinates for Toronto.

In [11]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Now to plot the neighborhoods on a map.

In [12]:
# create map of Downtown Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
count=0
# add markers to map
for lat, lng, label in zip(torontoCoords_df['Latitude'], torontoCoords_df['Longitude'], torontoCoords_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    count+=1
print("Number of Neighborhood markers: {}".format(count))
map_toronto

Number of Neighborhood markers: 103


## Now to only grab the neighborhoods from Boroughs with Toronto in the Borough Name.

The `torontoName_df` dataframe contains all the neighborhoods with `Toronto` in the borough name.

In [13]:
torontoName_df = torontoCoords_df[torontoCoords_df['Borough'].str.contains('Toronto')].reset_index(drop=True)

print("The number of Boroughs with Tornoto in their name: {}\n".format(len(torontoName_df['Borough'].unique())))
print("The number of neighborhoods within these {} Boroughs: {}\n".format(len(torontoName_df['Borough'].unique()), torontoName_df.shape[0]))
print("The Boroughs are:\n {}.\n".format((torontoName_df['Borough'].unique())))

print("The shape of the torontoName dataframe: {}\n".format(torontoName_df.shape))
torontoName_df.head()

The number of Boroughs with Tornoto in their name: 7

The number of neighborhoods within these 7 Boroughs: 39

The Boroughs are:
 ['East Toronto' 'East York/East Toronto' 'Central Toronto'
 'Downtown Toronto' 'Downtown Toronto Stn A' 'West Toronto'
 'East Toronto Business'].

The shape of the torontoName dataframe: (39, 5)



,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106
2,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
3,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
4,M4M,East Toronto,Studio District,43.659526,-79.340923


### Let's visualize the neighboorhoods within the `torontoName_df`.

In [14]:
# create map of Toronto using latitude and longitude values
map_torontoName = folium.Map(location=[latitude, longitude], zoom_start=11)
count=0
# add markers to map
for lat, lng, label in zip(torontoName_df['Latitude'], torontoName_df['Longitude'], torontoName_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_torontoName)  
    count+=1
print("Number of Neighborhood markers: {}".format(count))
map_torontoName

Number of Neighborhood markers: 39


## Listing Neighboorhoods and obtaining nearby venues of the `torontoName_df`.

Begin by setting up foursquare info

In [2]:
import creds
CLIENT_ID = creds.ClientID
CLIENT_SECRET = creds.ClientSecert
ACCESS_TOKEN = creds.AccessToken
VERSION = '20180604'
LIMIT = 30

In [6]:
CLIENT_ID

'ONTXUIG3XUMG0G2FSDQSGAFFV1MVHOZUZXHCF1WRFY1YQEXD'

### Define function for obtaining nearby Venues 

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
toronto_venues = getNearbyVenues(names=torontoName_df['Neighborhood'], latitudes=torontoName_df['Latitude'], longitudes=torontoName_df['Longitude'])

The Beaches
The Danforth  East
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Enclave of M5E
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction 

### Shape of  `toronto_venues` dataframe and its shape

In [18]:
print("Number of nearby venues to the neighboorhoods: ", toronto_venues.shape[0])
toronto_venues.head()

Number of nearby venues to the neighboorhoods:  833


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Danforth East,43.685347,-79.338106,Aldwych Park,43.684901,-79.341091,Park


### Let's check how many venues were returned for each neighborhood

In [19]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,30,30,30,30,30,30
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,30,30,30,30,30,30
Christie,16,16,16,16,16,16
Church and Wellesley,30,30,30,30,30,30
"Commerce Court, Victoria Hotel",30,30,30,30,30,30
Davisville,30,30,30,30,30,30
Davisville North,10,10,10,10,10,10


### Let's find out how many unique categories can be curated from all the returned venues

In [20]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 189 uniques categories.


## Now to Analyze Each Neighborhood

In [21]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print("toronto_onehot shape : ", toronto_onehot.shape)
toronto_onehot.head()

toronto_onehot shape :  (833, 189)


,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Escape Room,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Home Service,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts School,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [22]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print("toronoto_grouped new size: ", toronto_grouped.shape)
toronto_grouped.head()

toronoto_grouped new size:  (38, 189)


,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Escape Room,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Home Service,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts School,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.033333,0.000000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.033333,0.0,0.000000,0.033333,0.033333,0.0,0.0,0.033333,0.0000,0.0,0.0000,0.033333,0.0,0.000000,0.0,0.000000,0.0,0.0,0.033333,0.0,0.0000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.066667,0.066667,0.0,0.0,0.0,0.0,0.033333,0.000000,0.033333,0.000000,0.0,0.0,0.033333,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.033333,0.0,0.0,0.0,0.033333,0.033333,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.033333,0.0,0.0,0.000000,0.0,0.0,0.033333,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.033333,0.000000,0.0,0.0,0.0000,0.033333,0.0,0.0,0.0,0.000000,0.0000,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.033333,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.045455,0.0,0.045455,0.000000,0.000000,0.0,0.0,0.000000,0.0000,0.0,0.0000,0

### Let's print each neighborhood along with the top 5 most common venues

In [23]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                     venue  freq
0       Seafood Restaurant  0.07
1              Coffee Shop  0.07
2             Cocktail Bar  0.07
3           Farmers Market  0.07
4  Comfort Food Restaurant  0.03


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0                    Café  0.14
1             Coffee Shop  0.09
2          Breakfast Spot  0.09
3  Furniture / Home Store  0.05
4       Convenience Store  0.05


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3   Harbor / Marina  0.06
4               Bar  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.23
1                Café  0.07
2  Italian Restaurant  0.07
3         Yoga Studio  0.03
4           Gastropub  0.03


----Christie----
                venue  freq
0       Grocery St

### Let's put that into a _pandas_ dataframe

Function which sort venues in descending order


In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now creating a new dataframe which displays the top 10 venues for each neighborhood.

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

print("neighborhoods_venues_sorted new size: ", neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

neighborhoods_venues_sorted new size:  (38, 11)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Farmers Market,Bistro,Restaurant,Basketball Stadium,Beer Bar,Jazz Club,Breakfast Spot
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Convenience Store,Burrito Place,Nightclub,Restaurant,Italian Restaurant,Bar,Intersection
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Boutique,Rental Car Location,Boat or Ferry,Harbor / Marina,Coffee Shop,Airport Gate
3,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Spa,Modern European Restaurant,Miscellaneous Shop,Park,Middle Eastern Restaurant,Chinese Restaurant,Comic Shop
4,Christie,Grocery Store,Café,Park,Candy Store,Italian Restaurant,Baby Store,Restaurant,Athletics & Sports,Coffee Shop,Nightclub


## Cluster Neighborhoods


Run k-means to cluster the neighborhood into 5 clusters.

In [26]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 0, 3, 3, 0, 3, 0, 0, 0], dtype=int32)

In [27]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)



A new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [28]:
toronto_merged = torontoName_df

# merge toronto_grouped with torontoName_df to add latitude/longitude for each neighborhood
toronto_merged = pd.merge(torontoName_df, neighborhoods_venues_sorted, on='Neighborhood')
toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Health Food Store,Pub,Wine Bar,Creperie,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop
1,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106,1,Park,Convenience Store,Wine Bar,Cuban Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store
2,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Ice Cream Shop,Restaurant,Bookstore,Italian Restaurant,Yoga Studio,Pizza Place,Cosmetics Shop,Pub,Dessert Shop
3,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Sandwich Place,Fast Food Restaurant,Pizza Place,Ice Cream Shop,Steakhouse,Sushi Restaurant,Fish & Chips Shop,Liquor Store,Italian Restaurant,Gym
4,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,Café,American Restaurant,Bakery,Yoga Studio,Seafood Restaurant,Bookstore,Brewery,Cheese Shop,Pet Store


In [29]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining each of the clusters

### Cluster 1

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,Trail,Health Food Store,Pub,Wine Bar,Creperie,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop
2,"The Danforth West, Riverdale",Greek Restaurant,Ice Cream Shop,Restaurant,Bookstore,Italian Restaurant,Yoga Studio,Pizza Place,Cosmetics Shop,Pub,Dessert Shop
3,"India Bazaar, The Beaches West",Sandwich Place,Fast Food Restaurant,Pizza Place,Ice Cream Shop,Steakhouse,Sushi Restaurant,Fish & Chips Shop,Liquor Store,Italian Restaurant,Gym
4,Studio District,Coffee Shop,Café,American Restaurant,Bakery,Yoga Studio,Seafood Restaurant,Bookstore,Brewery,Cheese Shop,Pet Store
6,Davisville North,Food & Drink Shop,Park,Breakfast Spot,Gym,Gym / Fitness Center,Sandwich Place,Convenience Store,Department Store,Hotel,Dance Studio
7,North Toronto West,Clothing Store,Coffee Shop,Yoga Studio,Salon / Barbershop,Restaurant,Café,Chinese Restaurant,Park,Mexican Restaurant,Sporting Goods Shop
8,Davisville,Dessert Shop,Coffee Shop,Italian Restaurant,Sushi Restaurant,Café,Pizza Place,Sandwich Place,Gym,Diner,Indian Restaurant
9,"Summerhill West, Rathnelly, South Hill, Forest...",Coffee Shop,Fried Chicken Joint,Light Rail Station,Liquor Store,Restaurant,Bank,Bagel Shop,Supermarket,Sushi Restaurant,Pub
11,"St. James Town, Cabbagetown",Restaurant,Coffee Shop,Italian Restaurant,Café,Bakery,Park,Caribbean Restaurant,Jewelry Store,Japanese Restaurant,Deli / Bodega
12,Church and Wellesley,Park,Pub,Diner,Japanese Restaurant,Bookstore,Salon / Barbershop,Breakfast Spot,Restaurant,Bubble Tea Shop,Burger Joint


### Cluster 2


In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,The Danforth East,Park,Convenience Store,Wine Bar,Cuban Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store
10,Rosedale,Park,Playground,Trail,Wine Bar,Creperie,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store
23,Forest Hill North & West,Park,Jewelry Store,Trail,Sushi Restaurant,Wine Bar,Cuban Restaurant,Dog Run,Distribution Center,Discount Store,Diner


### Cluster 2


In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Roselawn,Pool,Garden,Home Service,Wine Bar,Creperie,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop


### Cluster 4


In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,"Regent Park, Harbourfront",Coffee Shop,Park,Café,Bakery,Breakfast Spot,Yoga Studio,Historic Site,Distribution Center,Restaurant,Spa
17,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Spa,Modern European Restaurant,Miscellaneous Shop,Park,Middle Eastern Restaurant,Chinese Restaurant,Comic Shop
18,"Richmond, Adelaide, King",Coffee Shop,Café,Restaurant,Asian Restaurant,Juice Bar,Seafood Restaurant,Monument / Landmark,Hotel,Smoke Shop,Bakery
20,"Toronto Dominion Centre, Design Exchange",Coffee Shop,Café,Restaurant,Japanese Restaurant,Gym,Bakery,Hotel,Steakhouse,Beer Bar,Pizza Place
21,"Commerce Court, Victoria Hotel",Café,Gastropub,Hotel,Coffee Shop,American Restaurant,Restaurant,Japanese Restaurant,Steakhouse,Tailor Shop,Bakery
24,"The Annex, North Midtown, Yorkville",Sandwich Place,Café,Coffee Shop,History Museum,Donut Shop,Burger Joint,Indian Restaurant,BBQ Joint,Pub,Middle Eastern Restaurant
25,"University of Toronto, Harbord",Café,Japanese Restaurant,Bookstore,Italian Restaurant,Bar,Bakery,Sushi Restaurant,Noodle House,Comfort Food Restaurant,Poutine Place
29,"First Canadian Place, Underground city",Café,Coffee Shop,Restaurant,Seafood Restaurant,Hotel,Gym,Japanese Restaurant,Sandwich Place,Pub,Bakery
30,Christie,Grocery Store,Café,Park,Candy Store,Italian Restaurant,Baby Store,Restaurant,Athletics & Sports,Coffee Shop,Nightclub
33,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Convenience Store,Burrito Place,Nightclub,Restaurant,Italian Restaurant,Bar,Intersection


### Cluster 5


In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Lawrence Park,Bus Line,Park,Swim School,Cuban Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store
